In [1]:
%reload_ext autoreload
%autoreload 2
os.chdir("..")
import toml
from core.dl_framework.utils import read_config

/usr/local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%%bash 
pwd

/content/Template_Project


In [7]:
config_file = toml.load("configs/default_train_config.toml")
setup_config = read_config(config_file)
from core.dl_framework.utils import get_specific_history

In [4]:
from core.dl_framework.learner import Learner
from core.dl_framework.callbacks import Recorder, Monitor, EarlyStopping, Checkpoints
from core.dl_framework.data import get_dataset, Dataset, DataLoader, split_data
from torchvision import transforms
bs = setup_config["h_batch_size"]
x_train, y_train, x_test, y_test = get_dataset(setup_config["s_source"], setup_config["s_set"], setup_config["p_tmp_data_path"], CNN=True)

train_ds, test_ds =  Dataset(x_train, y_train), Dataset(x_test, y_test)
train_dl, test_dl = DataLoader(train_ds, bs), DataLoader(test_ds, bs)
learn = Learner(train_ds, setup_config)
learn.fit(2)


100%|██████████| 187/187 [00:00<00:00, 1183.21it/s]


epoch: 1/2	[valid_acc: 0.8877	valid_loss: 0.7560	train_loss: 1.3755]
new best: valid_loss 0.7559569819088288
new checkpoint


100%|██████████| 187/187 [00:00<00:00, 1196.09it/s]

epoch: 2/2	[valid_acc: 0.9320	valid_loss: 0.4039	train_loss: 0.5547]
new best: valid_loss 0.4039432266179253
new checkpoint


In [5]:
from pathlib import Path
from datetime import datetime
ckp_path = "/content/gdrive/MyDrive/Marcel_Moczarski/Data_Science/Google_Services/Local_Drive_Storage/Conda_Envs/Template_Project/checkpoints"

In [169]:
sample = next(iter(test_dl))
imgs, targets = sample

In [39]:
best_hist = get_specific_history(Path(setup_config["c_c_ckp_path"]), "valid_loss", "parquet", "all")
best_hist[-1]

,valid_acc,valid_loss,train_loss
epochs,,,
1,0.894301,0.766493,1.393915e+00
2,0.933322,0.395780,5.485600e-01
3,0.950785,0.268248,3.294591e-01
4,0.960311,0.203980,2.377561e-01
5,0.965909,0.165154,1.866545e-01
...,...,...,...
196,0.990976,0.054431,3.345250e-07
197,0.990976,0.054759,3.018147e-07
198,0.990976,0.055092,2.723378e-07


In [40]:
# !pip install plotly
import plotly.io as pio
import itertools
pio.renderers.default = "notebook_connected"
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
col_pal = px.colors.sequential.Rainbow
col_pal_iter = itertools.cycle(col_pal) 
# def print_history(history):
history = best_hist
def plot_history(history):
    implemented_metric = ["acc", "loss"]
    plot_dict = {}
    for met in implemented_metric:
        plot_dict[met] = [s for s in history if met in s]

    fig = make_subplots(rows=1, cols=len(plot_dict), subplot_titles=list(plot_dict.keys()))

    for idx, (met, mets) in enumerate(plot_dict.items()):
        for key in mets:
            new_color = next(col_pal_iter)
            new_color = next(col_pal_iter)
            fig.add_trace(go.Scatter(x=history.index, y=history[key].values, name=key, line=dict(color=new_color)), row=1, col=idx+1)
            dmin, dmax = history[key].min(), history[key].max()
            if met == "acc":
                epoch = history[key].argmax()
                fig.add_trace(go.Scatter(x=[epoch], y=[dmax], name="best val", line=dict(color=new_color, width=1, dash="dash")), row=1, col=idx+1)
                fig.add_vline(x=epoch, line_width=1, line_color=new_color, line_dash="dash", row=1, col=idx+1)
            if met == "loss":
                epoch = history[key].argmin()
                fig.add_trace(go.Scatter(x=[epoch], y=[dmin], name="best val", line=dict(color=new_color, width=1, dash="dash")), row=1, col=idx+1)
                fig.add_vline(x=epoch, line_width=1, line_color=new_color, line_dash="dash", row=1, col=idx+1)
    # learn.history.keys()[0]
    # fig.update_layout(title_text="test")
    fig.show()
plot_history(best_hist[-1])